In [1]:
from collections import defaultdict
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#from pandas_summary import DataFrameSummary
import pickle

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## Create a data dictionary we can use for reference.

We load `data_dict.csv` to get the meaning and data type of each column, as well as possible value-label pairings.

In [2]:
dd = pd.read_csv("data_dict.csv")
dd = dd.drop(["NOTES", "SOURCE"], axis=1)
# clean up the column names
dd.columns = ["_".join(re.split("-| ", col.lower())) for col in dd.columns]
dd = dd.set_index("variable_name")
# remove all value-label info; we'll deal with that next
data_dict = dd[~pd.isna(dd.index)].drop(["value", "label"], axis=1)

In [3]:
data_dict.loc['UNITID']

name_of_data_element       Unit ID for institution
dev_category                                  root
developer_friendly_name                         id
api_data_type                              integer
Name: UNITID, dtype: object

## Create a mapping of integer factors to their actual meanings

In [4]:
label_dict = defaultdict(dict)
last_col = dd.index[0]
for row in dd.iterrows():
    index = row[0]
    if pd.isna(index):
        index = last_col
    else:
        last_col = index
    if not pd.isna(row[1].value):
        label_dict[index][row[1].value] = row[1].label

In [5]:
label_dict['PREDDEG']

{0.0: 'Not classified',
 1.0: 'Predominantly certificate-degree granting',
 2.0: "Predominantly associate's-degree granting",
 3.0: "Predominantly bachelor's-degree granting",
 4.0: 'Entirely graduate-degree granting'}

## First attept of choosing candidate columns

In [6]:
cols = [
    "UNITID",  # maybe concatenate with year
    "INSTNM",
    "STABBR",
    "ACCREDAGENCY",
    "HCM2",
    "MAIN",
    "NUMBRANCH",
    "PREDDEG",  # filter out 4
    "HIGHDEG",  # filter out 0, 1
    "CONTROL",
    "REGION",  # possibly instead of state
    "LOCALE",  # could backfill
   # "LOCALE2",  # was blank in 2017-18
    "CCBASIC",  # could backfill
    "CCUGPROF",  # could backfill
    "CCSIZSET",  # could backfill
    "HBCU",
    "MENONLY",
    "WOMENONLY",
    "RELAFFIL",  # could bin this
    "ADM_RATE",
    "SATVR25",
    "SATVR75",
    "SATMT25",
    "SATMT75",
    "SATWR25",
    "SATWR75",
    "SATVRMID",
    "SATMTMID",
    "SATWRMID",
    "ACTCM25",
    "ACTCM75",
    "ACTEN25",
    "ACTEN75",
    "ACTMT25",
    "ACTMT75",
    "ACTWR25",
    "ACTWR75",
    "ACTCMMID",
    "ACTENMID",
    "ACTMTMID",
    "ACTWRMID",
    "PCIP01",
    "PCIP03",
    "PCIP04",
    "PCIP05",
    "PCIP09",
    "PCIP10",
    "PCIP11",
    "PCIP12",
    "PCIP13",
    "PCIP14",
    "PCIP15",
    "PCIP16",
    "PCIP19",
    "PCIP22",
    "PCIP23",
    "PCIP24",
    "PCIP25",
    "PCIP26",
    "PCIP27",
    "PCIP29",
    "PCIP30",
    "PCIP31",
    "PCIP38",
    "PCIP39",
    "PCIP40",
    "PCIP41",
    "PCIP42",
    "PCIP43",
    "PCIP44",
    "PCIP45",
    "PCIP46",
    "PCIP47",
    "PCIP48",
    "PCIP49",
    "PCIP50",
    "PCIP51",
    "PCIP52",
    "PCIP54",
    "DISTANCEONLY",
    "UGDS",
    "PPTUG_EF",
    "CURROPER",  # filter
    "COSTT4_A",  # collapse with below
    "COSTT4_P",  # collapse with above
    "NUM4_PUB",  # collapse with below; use as filter
    "NUM4_PRIV",  # collapse with above; use as filter
    "TUITIONFEE_IN",  # use as flag for "in-state is different"?
    "TUITIONFEE_OUT",  # see above
    "TUITIONFEE_PROG",  # see above
    "INEXPFTE",
    "AVGFACSAL",
    "PFTFAC",
    "PCTPELL",
    "C150_4",
    "PCTFLOAN",
    "UG25ABV",
    "COMPL_RPY_1YR_RT",
    "COMPL_RPY_3YR_RT",
    "COMPL_RPY_5YR_RT",
    "COMPL_RPY_7YR_RT",
    "GRAD_DEBT_MDN",  # potential part of target
    "COUNT_NWNE_P6",  # potentially filter? /analysis
    "COUNT_WNE_P6",  # see above
    "MN_EARN_WNE_P6",  # target?
    "MD_EARN_WNE_P6",  # target 2?
    "ICLEVEL",  # potentially redundant
    "PRGMOFR",
]

In [7]:
print(f'{len(cols)} candidate columns')

108 candidate columns


## Make a version of our data dictionary just for the columns we're using

In [8]:
df_cols = pd.DataFrame(cols)
df_cols.columns = ["variable"]
df_cols["category"] = df_cols.variable.apply(lambda x: data_dict.loc[x, "dev_category"])
df_cols["type"] = df_cols.variable.apply(lambda x: data_dict.loc[x, "api_data_type"])
df_cols["friendly_name"] = df_cols.variable.apply(
    lambda x: data_dict.loc[x, "developer_friendly_name"]
)
df_cols["explanation"] = df_cols.variable.apply(
    lambda x: data_dict.loc[x, "name_of_data_element"]
)
df_cols.set_index("variable", inplace=True)
df_cols.head()

,category,type,friendly_name,explanation
variable,,,,
UNITID,root,integer,id,Unit ID for institution
INSTNM,school,autocomplete,name,Institution name
STABBR,school,string,state,State postcode
ACCREDAGENCY,school,string,accreditor,Accreditor for institution
HCM2,school,integer,under_investigation,Schools that are on Heightened Cash Monitoring...


## Ingest Data

In [9]:
dfs = dict()
year = 2001
while year < 2018:
    csv_name = f"data/MERGED{year}_{year-1999:02d}_PP.csv"
    print(f"loading {csv_name}")
    dfs[year] = pd.read_csv(
        csv_name, usecols=cols, na_values=["NU", "PrivacySuppressed"]
    )
    dfs[year].loc[:, "YEAR"] = year
    dfs[year].loc[:, "UNITID"] = (
        dfs[year].loc[:, "UNITID"].apply(lambda x: str(x) + "-" + str(year))
    )
    year += 1

loading data/MERGED2001_02_PP.csv
loading data/MERGED2002_03_PP.csv
loading data/MERGED2003_04_PP.csv
loading data/MERGED2004_05_PP.csv
loading data/MERGED2005_06_PP.csv
loading data/MERGED2006_07_PP.csv
loading data/MERGED2007_08_PP.csv
loading data/MERGED2008_09_PP.csv
loading data/MERGED2009_10_PP.csv
loading data/MERGED2010_11_PP.csv
loading data/MERGED2011_12_PP.csv
loading data/MERGED2012_13_PP.csv
loading data/MERGED2013_14_PP.csv
loading data/MERGED2014_15_PP.csv
loading data/MERGED2015_16_PP.csv
loading data/MERGED2016_17_PP.csv
loading data/MERGED2017_18_PP.csv


## Drop null columns

In [10]:
df = pd.concat(dfs.values()).set_index('UNITID')

In [11]:
old_cols = df.columns
new_cols = df.dropna(axis="columns", how="all").columns
#df = df[new_cols].set_index("UNITID")
df_cols = df_cols.loc[df.columns]
label_dict_old = label_dict
label_dict = {
    key: label_dict_old[key]
    for key in (set(df.columns).intersection(set(label_dict_old.keys())))
}
print(f"{len(list(set(old_cols) - set(new_cols)))} columns dropped")

0 columns dropped


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  after removing the cwd from sys.path.


In [12]:
top_earners = df.sort_values("MN_EARN_WNE_P6", ascending=False)[
    ["INSTNM", "MN_EARN_WNE_P6"]
].dropna()
top_earners

,INSTNM,MN_EARN_WNE_P6
UNITID,,
215123-2014,Philadelphia College of Osteopathic Medicine,151500.0
239169-2011,Medical College of Wisconsin,151000.0
207315-2009,Oklahoma State University Center for Health Sc...,142800.0
239169-2012,Medical College of Wisconsin,138400.0
239169-2013,Medical College of Wisconsin,128100.0
...,...,...
242954-2007,Modern Hairstyling Institute-Arecibo,11000.0
158282-2005,Bastrop Beauty School Inc,11000.0
241906-2012,Emma's Beauty Academy-Mayaguez,10700.0


In [13]:
df[df['INSTNM'] == 'Hofstra University']['MD_EARN_WNE_P6']

UNITID
191649-2001        NaN
191649-2002        NaN
191649-2003    42400.0
191649-2004        NaN
191649-2005    42600.0
191649-2006        NaN
191649-2007    43500.0
191649-2008        NaN
191649-2009    40300.0
191649-2010        NaN
191649-2011    37700.0
191649-2012    36700.0
191649-2013    38300.0
191649-2014    39900.0
191649-2015        NaN
191649-2016        NaN
191649-2017        NaN
Name: MD_EARN_WNE_P6, dtype: float64

In [14]:
i = 1
df_cols[i * 10 : i * 10 + 10]

,category,type,friendly_name,explanation
LOCALE,school,integer,locale,Locale of institution
CCBASIC,school,integer,carnegie_basic,Carnegie Classification -- basic
CCUGPROF,school,integer,carnegie_undergrad,Carnegie Classification -- undergraduate profile
CCSIZSET,school,integer,carnegie_size_setting,Carnegie Classification -- size and setting
HBCU,school,integer,minority_serving.historically_black,Flag for Historically Black College and Univer...
MENONLY,school,integer,men_only,Flag for men-only college
WOMENONLY,school,integer,women_only,Flag for women-only college
RELAFFIL,school,integer,religious_affiliation,Religous affiliation of the institution
ADM_RATE,admissions,float,admission_rate.overall,Admission rate
SATVR25,admissions,float,sat_scores.25th_percentile.critical_reading,25th percentile of SAT scores at the instituti...


In [15]:
backfill = defaultdict(dict)
backfill_cols = [
    "LOCALE",
    "CCBASIC",
    "CCUGPROF",
    "CCSIZSET",
    "CURROPER",
    "NUM4_PUB",
    "NUM4_PRIV",
]
for row in df.query("YEAR==2017")[["INSTNM", *backfill_cols]].iterrows():
    for col in backfill_cols:
        backfill[row[1]["INSTNM"]][col] = row[1][col]

In [16]:
for col in backfill_cols:
    df.loc[:, col] = df.apply(lambda x: backfill[x["INSTNM"]].get(col), axis=1)

In [17]:
#cols_drop_pre = ["INSTNM"]
cols_drop_pre = []
categorical_keys = [
    "STABBR",
    "ACCREDAGENCY",
    "PREDDEG",
    "HIGHDEG",
    "CONTROL",
    "REGION",
    "LOCALE",
    "CCBASIC",
    "CCUGPROF",
    "CCSIZSET",
    "RELAFFIL",
   # "ICLEVEL",
]
cols_drop_post = ["CURROPER", "COSTT4_A", "COSTT4_P", "NUM4_PUB", "NUM4_PRIV", "NUM4"]

on_hold = [
#    "TUITIONFEE_IN",
#    "TUITIONFEE_OUT",
#    "TUITIONFEE_PROG",
    "MN_EARN_WNE_P6",
    "COUNT_NWNE_P6",
#    "COUNT_WNE_P6"
    "ICLEVEL",
]

In [18]:
sum(pd.notna(df['COUNT_WNE_P6']))

48735

In [19]:
# drop columns deemed unneeded
X = df.drop(cols_drop_pre, axis=1)
# must have at least 50 undergrads
X = X.query("UGDS >= 50")
X = X.query("CURROPER == 1")
X = X.query("PREDDEG != 4")
X = X.query("HIGHDEG not in [0,1]")
X["COSTT4_A"].fillna(value=X["COSTT4_P"], inplace=True)
X["COSTT4_P"].fillna(value=X["COSTT4_A"], inplace=True)
X.loc[:, "COSTT4"] = X.loc[:, "COSTT4_P"]
X["NUM4_PRIV"].fillna(value=X["NUM4_PUB"], inplace=True)
X["NUM4_PUB"].fillna(value=X["NUM4_PRIV"], inplace=True)
X.loc[:, "NUM4"] = X.loc[:, "NUM4_PUB"]
X = X.query("NUM4 > 250")
X = X.drop(cols_drop_post, axis=1)
X = X.drop(on_hold, axis=1)
print(X.YEAR.value_counts())

2017    1582
2016    1579
2015    1578
2014    1572
2013    1570
2012    1563
2011    1560
2009    1554
2010    1553
2008    1550
2007    1550
2006    1548
2004    1539
2005    1538
2003    1536
2002    1531
2001    1529
Name: YEAR, dtype: int64


In [20]:
regions = [v.split("(")[0].rstrip() for v in label_dict["REGION"].values()]

In [21]:
X["REGION"] = X.REGION.apply(lambda x: label_dict["REGION"][x].split("(")[0])

In [22]:
X["CONTROL"] = X.CONTROL.apply(lambda x: label_dict["CONTROL"][x])

In [23]:
X["MAIN"] = X.MAIN.apply(lambda x: label_dict["MAIN"][x])

In [24]:
X["HIGHDEG"] = X.HIGHDEG.apply(lambda x: label_dict["HIGHDEG"][x].split(" ")[0])

In [25]:
X["PREDDEG"] = X.PREDDEG.apply(
    lambda x: ["not_classified", "certificate", "associate", "bachelor", "grad"][x]
)

In [26]:
X.columns

Index(['INSTNM', 'STABBR', 'ACCREDAGENCY', 'HCM2', 'MAIN', 'NUMBRANCH',
       'PREDDEG', 'HIGHDEG', 'CONTROL', 'REGION',
       ...
       'COMPL_RPY_1YR_RT', 'COMPL_RPY_3YR_RT', 'COMPL_RPY_5YR_RT',
       'COMPL_RPY_7YR_RT', 'GRAD_DEBT_MDN', 'COUNT_WNE_P6', 'MD_EARN_WNE_P6',
       'PRGMOFR', 'YEAR', 'COSTT4'],
      dtype='object', length=101)

In [27]:
df_sum = DataFrameSummary(X)
df_sum.columns_stats

NameError: name 'DataFrameSummary' is not defined

In [ ]:
with open("raw_df.pickle", "wb") as f:
    pickle.dump(X, f)